In [1]:
import os
import shutil
import logging
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import hydra
import numpy as np
import openpack_toolkit as optk
import openpack_torch as optorch
import pandas as pd
from omegaconf import DictConfig, OmegaConf

from openpack_toolkit import OPENPACK_OPERATIONS

C:\Users\Ego\anaconda3\envs\pytorchenv3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
optorch.configs.register_configs()

In [3]:
with hydra.initialize_config_dir(version_base=None, config_dir="C:\\Users\\Ego\\Documents\\TFM\\repo\\tfm1\\config"):
    cfg = hydra.compose(
        # config_name="unet.yaml",
        config_name="config.yaml",
    )    
cfg.dataset.annotation.activity_sets = dict() # Remove this attribute just for the simpler visualization.
cfg.dataset.split = optk.configs.datasets.splits.DEBUG_SPLIT

In [4]:
print(OmegaConf.to_yaml(cfg.dataset.stream))

name: all-devices
description: All devices stream.
super_stream: null
path_imu:
  dir: ${path.openpack.rootdir}/${user.name}/atr/${device}
  fname: ${session}.csv
frame_rate_imu: 30
devices:
- atr01
- atr02
- atr03
- atr04
acc: true
gyro: false
quat: false
imu_dim: 12
min_value_imu_acc: -4
max_value_imu_acc: 4
min_value_imu_gyro: -2000
max_value_imu_gyro: 2000
path_keypoint:
  dir: ${path.openpack.rootdir}/${user.name}/kinect/${..category}/${..model}/single
  fname: ${session}.json
frame_rate_keypoint: 15
category: 2d-kpt
model: mmpose-hrnet-w48-posetrack18-384x288-posewarper-stage2
nodes:
  0: nose
  1: left_eye
  2: right_eye
  3: left_ear
  4: right_ear
  5: left_shoulder
  6: right_shoulder
  7: left_elbow
  8: right_elbow
  9: left_wrist
  10: right_wrist
  11: left_hip
  12: right_hip
  13: left_knee
  14: right_knee
  15: left_ankle
  16: right_ankle
keypoint_dim: 34
min_value_keypoint: 0
max_value_keypoint: 1280



In [6]:
import sys
sys.path.append('../')
from utils.datasets import OpenPackAll
class OpenPackAllDataModule(optorch.data.OpenPackBaseDataModule):
    dataset_class = OpenPackAll

    def get_kwargs_for_datasets(self, stage: Optional[str] = None) -> Dict:
        kwargs = {
            "window": self.cfg.train.window,
            "debug": self.cfg.debug,
        }
        return kwargs


In [7]:
datamodule = OpenPackAllDataModule(cfg)
datamodule.setup("test")
dataloaders = datamodule.test_dataloader()

batch = dataloaders[0].dataset.__getitem__(0)
print(batch.keys())

Shape of x keypoints: (2, 27377, 17)
Shape of t keypoints: (27377,)
Shape of x imu: (12, 55272)
Shape of t imu: (55272,)


TypeError: OpenPackAll.create_pd_from_data() takes 2 positional arguments but 3 were given

In [ ]:
datamodule.